In [34]:
from transformers.models.bloom.modeling_bloom import BloomModel
NAME = "bigscience/bloom-560m"

from slicer_wrapper import SlicingConfig, tensor_parallel

In [35]:
slicing_config = SlicingConfig(
    {
        "self_attention.query_key_value": "vertical",
        "self_attention.dense": "horizontal",
        "mlp.dense_h_to_4h": "vertical",
        "mlp.dense_4h_to_h": "horizontal",
    },
    {
        "BloomAttention": {"input": {"alibi": "cut", 1: "scale"}, "output": {0: "reduce"}, "attributes": {"num_heads": "scale_int"}},
        "BloomMLP": {"input": {1: "scale"}, "output": {"ALL": "reduce"}, "attributes":{}},
    },
)

In [38]:
model = tensor_parallel(BloomModel, slicing_config, 0, 2).from_pretrained(NAME)

In [39]:
model

BloomModel(
  (word_embeddings): Embedding(250880, 1024)
  (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (h): ModuleList(
    (0): BloomBlock(
      (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (self_attention): ParallelLayerWrapper(
        (module): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (mlp): ParallelLayerWrapper(
        (module): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (1): BloomBlock(
      (input_layernorm):